In [36]:
import numpy as np
import pandas as pd
import time as dt

In [126]:
def conver_time(timestr):
    ts,ms = timestr.split('.')
    timest = dt.strptime(ts, '%Y-%m-%d %H:%M:%S')
    return int(1000*(dt.mktime(timest)+10800)+int(ms))
vconv_time = np.vectorize(conver_time)

In [138]:
data = pd.read_csv('data-train.csv')
x = np.array((data[['ID', 'Timestamp', 'Location']]))
x[:,1] = vconv_time(x[:,1])
y = np.array(data['Category'])
n = x.shape[1]
m = y.size